In [1]:
import os
import keras_ocr
import random
from PIL import Image, ImageDraw, ImageFont

2024-09-26 04:52:31.982729: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-26 04:52:32.152954: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
pipeline = keras_ocr.pipeline.Pipeline()

Looking for /root/.keras-ocr/craft_mlt_25k.h5


2024-09-26 04:52:35.479761: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-26 04:52:37.804944: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 19518 MB memory:  -> device: 0, name: NVIDIA RTX A5000, pci bus id: 0000:01:00.0, compute capability: 8.6
2024-09-26 04:52:37.805775: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22333 MB memory:  -> device: 1, name: NVIDIA RTX A5000, pci bus id: 0000:24:00.0, compute capability: 8.6
2024-09-26 04:52:37.806426: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:2 w

Looking for /root/.keras-ocr/crnn_kurapan.h5


In [3]:
data_dir = 'text'

wordlist_file = os.path.join(data_dir, 'sah.wordlist')
training_text_file = os.path.join(data_dir, 'sah.training_text')
fonts_list_file = os.path.join(data_dir, 'okfonts_test.txt')
fonts_dir = os.path.join(data_dir, 'fonts')

output_dir = 'generated_images'

In [4]:
with open(wordlist_file, 'r', encoding='utf-8') as f:
    words = [line.strip() for line in f.readlines()]

with open(fonts_list_file, 'r', encoding='utf-8') as f:
    styles = [line.strip() for line in f.readlines()]

In [5]:
def generate_image(text, font_path, image_size=(500, 100), font_size=30):
    image = Image.new('RGB', image_size, color=(255, 255, 255))
    draw = ImageDraw.Draw(image)
    try:
        font = ImageFont.truetype(font_path, font_size)
    except IOError:
        print(f"Не удалось загрузить шрифт: {font_path}")
        return None
        
    # Определение границ текста
    text_bbox = draw.textbbox((0, 0), text, font=font)
    text_width = text_bbox[2] - text_bbox[0]
    text_height = text_bbox[3] - text_bbox[1]

    # Центрирование текста
    text_x = (image_size[0] - text_width) // 2
    text_y = (image_size[1] - text_height) // 2

    draw.text((text_x, text_y), text, font=font, fill=(0, 0, 0))
    
    return image

In [6]:
def load_fonts_from_styles(styles_list, base_fonts_dir):
    fonts = []
    for style in styles_list:
        style_dir = os.path.join(base_fonts_dir, style)
        if os.path.exists(style_dir):
            for file in os.listdir(style_dir):
                if file.endswith(('.ttf', '.otf')):
                    fonts.append(os.path.join(style_dir, file))
        else:
            print(f"Директория для стиля '{style}' не найдена: {style_dir}")
    return fonts

In [7]:
fonts = load_fonts_from_styles(styles, fonts_dir)

In [8]:
for word in words:
    random_font = random.choice(fonts)

    output_path = os.path.join(output_dir, f'{word}.png')
    
    image = generate_image(word, random_font)
    
    if image:
        image.save(output_path)
        print(f"Сгенерировано изображение для слова: {word}")

In [9]:
images = [keras_ocr.tools.read(os.path.join(output_dir, f"{word}.png")) for word in words]
labels = [[(word, None)] for word in words]

In [10]:
train_images, test_images = images[:80], images[80:]
train_labels, test_labels = labels[:80], labels[80:]

In [11]:
detector = keras_ocr.detection.Detector()

detector.fit(images=train_images, targets=train_labels, epochs=10)

Looking for /root/.keras-ocr/craft_mlt_25k.h5


AttributeError: 'Detector' object has no attribute 'fit'